In [ ]:
%load_ext autoreload
%autoreload 2
%aimport theme
import pandas as pd
import altair as alt
import pygwalker as pyg
from altair import datum
from theme import apply_theme
alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
COLORS = {
    'd': '#56B4E9',
    'j': '#CC79A7',
    'g': '#009E73'
}
GOV_EFR = 0.015346

In [ ]:
df = pd.read_csv('../input/Nov-21-2023/3-23-2024_failure_rate_meta.csv')
df

In [ ]:
df_overall = df[
    (df.continent == 'ALL') &
    (df.country == 'ALL') &
    (df.publisher == 'ALL') |
    (df.web_type == 'government') &
    (df.publisher == 'ALL')
]

df_overall.web_type = df_overall.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')

df_overall

In [ ]:
base = alt.Chart(df_overall).mark_circle(
    size=60,
    opacity=1
).encode(
    alt.Y('web_type:N', title=None, sort=['US Government Websites', 'Journal Websites', 'Data Portals']),
    alt.X('fail_rate_meta:Q', title='Estimated failure rate').axis(format='.0%'),
    alt.Color('web_type:N').scale(range=list(COLORS.values())).legend(None)
).properties(
    height=200,
    width=300
)

error = base.mark_errorbar(
    thickness=2,
    color='black'
).encode(
    alt.X('ci_95L', title='Estimated failure rate'),
    alt.X2('ci_95U'),
    color=alt.value('black')
)

plot = base + error + base

plot = apply_theme(plot)

plot

In [ ]:
df_journal_continent = df[(df.web_type != 'government') & (df.country != 'ALL') & (df.country.notnull())]

In [ ]:
countries_data_portals = [
    'Spain', 'China', 'Germany', 'India', 'Italy', 'Japan', 'Canada', 'France', 'United States', 'United Kingdom'
]
countries_journals = [
    'China', 'Germany', 'India', 'Spain', 'Italy', 'Switzerland', 'Netherlands', 'France', 'United States', 'United Kingdom'
]
df_journal_continent_filtered = df_journal_continent[
    (df_journal_continent.web_type == 'data_portal') & (df_journal_continent.country.isin(countries_data_portals))|
    (df_journal_continent.web_type == 'journal') & (df_journal_continent.country.isin(countries_journals))
]

In [ ]:
df_journal_continent_filtered.web_type = df_journal_continent_filtered.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')

In [ ]:
base = alt.Chart(df_journal_continent_filtered).mark_circle(size=100, opacity=1).encode(
    alt.Y('country:N', title=None).sort(field="fail_rate_meta", op="max", order="descending"),
    alt.X('fail_rate_meta:Q', title='Estimated failure rate').axis(format='.0%'),
    alt.Color('web_type:N').scale(range=list(COLORS.values())).legend(None),
    # alt.Size('units:Q')
    # alt.Column('web_type:N')
).properties(
    height=300
)

error = base.mark_errorbar(

).encode(
    alt.X('ci_95L:Q', title='Estimated failure rate'),
    alt.X2('ci_95U:Q')
)

text = base.mark_text(
    dx=15,
    dy=-9
).encode(
    alt.Text('units'),
    color=alt.value('black')
)

gov = base.mark_rule(
    color='black',
    size=1,
    strokeDash=[4, 4]
).encode(
    alt.Y(),
    alt.X(f'baseline:Q', title='Estimated failure rate'),
    alt.Size(),
    alt.YOffset(),
    color=alt.value('black')
).transform_calculate(
    baseline=f"{GOV_EFR}"
)

plot = (base + error + text + gov)

plot = (
    plot.transform_filter("datum.web_type == 'Data Portals'").properties(title='Data Portals') |
    plot.transform_filter("datum.web_type == 'Journal Websites'").properties(title='Journal Websites')
)

# .facet(
#     alt.Column('web_type:N', title=None)
# ).resolve_scale(y='independent')

plot = apply_theme(plot)

plot

In [ ]:
df_continents = df[(df.web_type != 'government') & (df.continent != 'ALL') & (df.continent.notnull())]
df_continents.web_type = df_continents.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')
df_continents

In [ ]:
base = alt.Chart(df_continents).mark_circle(size=100, opacity=1).encode(
    alt.Y('continent:N', title=None).sort(field="fail_rate_meta", op="max", order="descending"),
    alt.X('fail_rate_meta:Q', title='Estimated failure rate').axis(format='.0%'),
    alt.Color('web_type:N').scale(range=list(COLORS.values())).legend(None),
    # alt.Size('units:Q')
    # alt.Column('web_type:N')
).properties(
    height=300
)

error = base.mark_errorbar(

).encode(
    alt.X('ci_95L:Q', title='Estimated failure rate'),
    alt.X2('ci_95U:Q')
)

text = base.mark_text(
    dx=15,
    dy=-9
).encode(
    alt.Text('units'),
    color=alt.value('black')
)

gov = base.mark_rule(
    color='black',
    size=1,
    strokeDash=[4, 4]
).encode(
    alt.Y(),
    alt.X(f'baseline:Q', title='Estimated failure rate'),
    alt.Size(),
    alt.YOffset(),
    color=alt.value('black')
).transform_calculate(
    baseline=f"{GOV_EFR}"
)

plot = (base + error + text + gov)

plot = (
    plot.transform_filter("datum.web_type == 'Data Portals'").properties(title='Data Portals') |
    plot.transform_filter("datum.web_type == 'Journal Websites'").properties(title='Journal Websites')
)

# .facet(
#     alt.Column('web_type:N', title=None)
# ).resolve_scale(y='independent')

plot = apply_theme(plot)

plot

In [ ]:
df_org = df[(df.web_type != 'government') & (df.publisher != 'ALL') & (df.publisher.notnull())]
df_org.web_type = df_org.web_type.apply(lambda x: 'Journal Websites' if x == 'journal' else 'US Government Websites' if x == 'government' else 'Data Portals')
df_org

In [ ]:
top_10_journal_publishers_by_size = df_org[(df_org.web_type == 'Journal Websites') & (df_org.units > 150)]

In [ ]:
top_10_data_portal_publishers_by_size = df_org[(df_org.web_type == 'Data Portals') & (df_org.units > 27)]

In [ ]:
df_org_filtered = pd.concat([top_10_journal_publishers_by_size, top_10_data_portal_publishers_by_size])
df_org_filtered

In [ ]:
base = alt.Chart(df_org_filtered).mark_circle(size=100, opacity=1).encode(
    alt.Y('publisher:N', title=None).sort(field="fail_rate_meta", op="max", order="descending"),
    alt.X('fail_rate_meta:Q', title='Estimated failure rate').axis(format='.0%'),
    alt.Color('web_type:N').scale(range=list(COLORS.values())).legend(None),
    # alt.Size('units:Q')
    # alt.Column('web_type:N')
).properties(
    height=300
)

error = base.mark_errorbar(

).encode(
    alt.X('ci_95L:Q', title='Estimated failure rate'),
    alt.X2('ci_95U:Q')
)

text = base.mark_text(
    dx=15,
    dy=-9
).encode(
    alt.Text('units'),
    color=alt.value('black')
)

gov = base.mark_rule(
    color='black',
    size=1,
    strokeDash=[4, 4]
).encode(
    alt.Y(),
    alt.X(f'baseline:Q', title='Estimated failure rate'),
    alt.Size(),
    alt.YOffset(),
    color=alt.value('black')
).transform_calculate(
    baseline=f"{GOV_EFR}"
)

plot = (base + error + text + gov)

plot = (
    plot.transform_filter("datum.web_type == 'Data Portals'").properties(title='Data Portals') |
    plot.transform_filter("datum.web_type == 'Journal Websites'").properties(title='Journal Websites')
)

# .facet(
#     alt.Column('web_type:N', title=None)
# ).resolve_scale(y='independent')

plot = apply_theme(plot)

plot